In [1]:
import rasterio
import numpy as np
from rasterio.mask import mask
import fiona
from shapely.geometry import shape

In [2]:
def calculate_volume_rasterio(dem_path, shapes, base_level=0):
    try:
        # Open the DEM file
        with rasterio.open(dem_path) as src:
            # Mask the DEM using provided polygon geometry (shapes)
            out_image, out_transform = mask(src, shapes, crop=True)
            out_image = out_image[0]  # Get the first band

            # Mask out NoData values
            dem_array = np.ma.masked_equal(out_image, src.nodata)

            # Calculate pixel area (assuming square pixels)
            pixel_size = src.res[0]  # Assuming square pixels
            pixel_area = pixel_size ** 2

            # Calculate the volume above and below a base level
            fill_volume = np.sum(np.maximum(dem_array - base_level, 0)) * pixel_area
            cut_volume = np.sum(np.maximum(base_level - dem_array, 0)) * pixel_area
            net_volume = fill_volume - cut_volume

            return fill_volume, cut_volume, net_volume

    except Exception as e:
        print(f"Error in volume calculation: {str(e)}")
        return None

In [3]:
# Function to read a polygon from a shapefile
def get_polygon_from_shapefile(shapefile_path):
    with fiona.open(shapefile_path, "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]  # Get all polygons
        return shapes

In [4]:
shapefile_path = "C:/Users/vaish/Downloads/Aereo1/heap/polygons.shp"
dem_file = "C:/Users/vaish/Downloads/Aereo1/heap-dem.tif"

In [5]:
# Get the polygon geometry from the shapefile
polygon_shapes = get_polygon_from_shapefile(shapefile_path)

In [6]:
# Check CRS of DEM
with rasterio.open(dem_file) as dem_src:
    print("DEM CRS:", dem_src.crs)

# Check CRS of shapefile
with fiona.open(shapefile_path, "r") as shp:
    print("Shapefile CRS:", shp.crs)

DEM CRS: EPSG:32644
Shapefile CRS: EPSG:4326


In [7]:
import geopandas as gpd
from fiona.crs import from_epsg

# Load shapefile as GeoDataFrame
shapefile_gdf = gpd.read_file(shapefile_path)

# Reproject the shapefile to the DEM's CRS
dem_crs = rasterio.open(dem_file).crs
shapefile_reprojected = shapefile_gdf.to_crs(dem_crs)

# Convert back to GeoJSON-like format (for rasterio)
reprojected_shapes = [shape(geom) for geom in shapefile_reprojected.geometry]

In [8]:
# Calculate the volumes
fill_volume, cut_volume, net_volume = calculate_volume_rasterio(dem_file, reprojected_shapes)
print(f"Fill Volume: {fill_volume} cubic meters")
print(f"Cut Volume: {cut_volume} cubic meters")
print(f"Net Volume: {net_volume} cubic meters")

Fill Volume: 169220.04217865225 cubic meters
Cut Volume: 0.0 cubic meters
Net Volume: 169220.04217865225 cubic meters
